In [2]:
from dotenv import load_dotenv
load_dotenv() ## load all the nevironment variables
import os
# from googleapiclient import build
import requests
import json
from bs4 import BeautifulSoup
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build
import pandas as pd
import scrapetube
import time
# from youtube_api import YoutubeDataApi

In [3]:
%pwd

'd:\\Projects\\abit\\script_generator\\notebooks'

In [4]:
os.chdir('../')

In [2]:
# channel_name = 'The ArmchairHistorian'# user input 
channel_name = input()# user input 
print(channel_name)
# youtube_api_key = os.getenv("YOUTUBE_API_KEY")

foxhollowgames


In [ ]:
# Psych2go
# Salim_Ahmed
# Marcus-Jones

In [ ]:
video_path = "data\\video_data\\"
transcripts_path = "data\\transcripts_data\\"

In [5]:
channel_name='Psych2go'

In [6]:
error_video_lists=[]
x=[]
y=['video_id','video_title','publish_time','total_views','video_length','video_description']

In [7]:
videos = scrapetube.get_channel(  
    # channel_id = channel_id,
    # channel_url: str = None,
    channel_username = channel_name,
    # limit = 1,
    sleep = 1,
    # proxies: dict = None,
    sort_by = "newest",                  #     Literal["newest", "oldest", "popular"]
    content_type = "videos"   #    Literal["videos", "shorts", "streams"]
    )

for video in videos:
   
    # print(video)
    # y=video.copy()
    # print(video['publishedTimeText']['simpleText'])
    # sample=video.copy()
    video_id=video['videoId']
    video_title=video['title']['runs'][0]['text']
    video_description = video['descriptionSnippet']['runs'][0]['text']
    video_length = video['lengthText']['simpleText']
    # print(video_title)
    publish_time=video['publishedTimeText']['simpleText']
    try:
        total_views=video['viewCountText']['simpleText']
    except:
        print(video_title)
        error_video_lists.append(video_title)

        total_views = 'PREMIUM'
    x.append([video_id,video_title,publish_time,total_views,video_length,video_description])
    
    # print(video['title'])
    # print(video.values())
    # print(video['videoId'])

    # video_ids.append(video['videoId'])
    

In [ ]:
video_df=pd.DataFrame(data=x,columns=y)

In [ ]:
video_df.to_csv(f"data\\video_data\\{channel_name}_video_data.csv")

In [ ]:
# publish_time = '1 month ago'

In [55]:
def filter_time(df, publish_time):
    condition = df['publish_time'] == publish_time
    df = df.loc[:condition.idxmax()]
    return df

In [ ]:
video_df = filter_time(video_df,publish_time)

In [44]:
def convert_to_minutes(time_list):
    minutes_list = []
    for time_str in time_list:
        # Split the time string into components
        parts = list(map(int, time_str.split(':')))
        
        # Initialize total_minutes
        total_minutes = 0
        
        # Calculate total minutes based on the number of parts
        if len(parts) == 3:  # Format is "hours:minutes:seconds"
            hours, minutes, seconds = parts
            total_minutes = hours * 60 + minutes + seconds / 60
        elif len(parts) == 2:  # Format is "minutes:seconds"
            minutes, seconds = parts
            total_minutes = minutes + seconds / 60
        elif len(parts) == 1:  # Format is "seconds"
            total_minutes = parts[0] / 60
        
        # Append the result to the list
        minutes_list.append(round(total_minutes,2))
    
    return minutes_list


In [46]:
video_df['video_length_mins']=convert_to_minutes(video_df['video_length'])

C:\Users\Pakshal Jain\AppData\Local\Temp\ipykernel_5548\1022202905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['video_length_mins']=convert_to_minutes(subset_df['video_length'])


In [23]:
def view_split(x):
    x=int(x.split(' ')[0].replace(',',''))
    return x

In [25]:
video_df['view_count']=video_df['total_views'].apply(view_split)

C:\Users\Pakshal Jain\AppData\Local\Temp\ipykernel_5548\3769320690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df['view_count']=video_df['total_views'].apply(view_split)


In [91]:
# for video in videos:
#     # print(video)
#     # y=video.copy()
#     # print(video['publishedTimeText']['simpleText'])
#     video_id=video['videoId']
#     video_title=video['title']['runs']['text']
#     publish_time=video['publishedTimeText']['simpleText']
#     total_views=video['viewCountText']['simpleText']
#     print(video['title'])
#     # print(video.values())
#     # print(video['videoId'])
#     # video_ids.append(video['videoId'])

In [27]:
video_ids=video_df['video_id'].tolist()

In [50]:
def extract_transcript_details(video_ids):
    transcript_list=[]
    errors_list=[]
    for video_id in video_ids:
        try:
            transcript_text = YouTubeTranscriptApi.get_transcript(video_id)

            transcript = ""
            for i in transcript_text:
                transcript += " " + i["text"]

            transcript_list.append(transcript)
            # return transcript

        except Exception as e:
            print(e)
            transcript_list.append('No Transcript')
            print(video_id)
            errors_list.append(video_id)
    return transcript_list

            # pass
        # raise e

In [51]:
start=time.time()
transcripts_list = extract_transcript_details(video_ids)
end=time.time()
print((end-start)/60)

1.2688398281733195


In [52]:
video_df['transcripts'] = transcripts_list
video_df['character_count']=video_df['transcripts'].str.len()
video_df['word_count']=video_df['transcripts'].str.split().apply(len)

C:\Users\Pakshal Jain\AppData\Local\Temp\ipykernel_5548\374399334.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df['transcripts'] = transcripts_list
C:\Users\Pakshal Jain\AppData\Local\Temp\ipykernel_5548\374399334.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df['character_count']=video_df['transcripts'].str.len()
C:\Users\Pakshal Jain\AppData\Local\Temp\ipykernel_5548\374399334.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [54]:
video_df.to_csv(f"{transcripts_path}{channel_name}_transcripts.csv")